In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd

In [ ]:
# Load the data from 'signal.csv'
# Load your CSV file into a pandas DataFrame
df = pd.read_csv('XAU_USD_2023_MULTICLASS.csv', parse_dates=['datetime'], index_col='datetime')

In [ ]:
df

,Volume,Open,High,Low,Close,signal
datetime,,,,,,
2023-09-12 19:00:00+00:00,773,1913.430,1913.540,1913.095,1913.200,0
2023-09-12 19:15:00+00:00,705,1913.185,1913.665,1912.975,1913.520,2
2023-09-12 19:30:00+00:00,694,1913.525,1913.565,1912.685,1912.810,1
2023-09-12 19:45:00+00:00,919,1912.800,1913.405,1912.800,1913.060,0
2023-09-12 20:00:00+00:00,464,1912.995,1913.356,1912.815,1913.025,0
...,...,...,...,...,...,...
2023-12-29 20:45:00+00:00,1649,2064.350,2064.555,2062.710,2063.230,1
2023-12-29 21:00:00+00:00,792,2063.280,2063.710,2062.710,2062.940,1
2023-12-29 21:15:00+00:00,525,2062.935,2063.300,2062.625,2062.715,0


In [ ]:
from backtesting import Strategy

class SignalStrategy(Strategy):
    risk_per_trade = 0.01  # Risk 1% of equity per trade
    reward_to_risk_ratio = 1.0  # Take profit is the same distance as risk

    def init(self):
        # Convert the signal column to a proper pandas Series for backtesting.py
        self.signal = self.I(lambda x: x, self.data.signal)

    def next(self):
        if self.signal[-1] == 2:  # If the signal is a buy
            if not self.position:  # And we do not already have an open position
                self.enter_long()   # Enter a long position
        elif self.signal[-1] == 1:  # If the signal is a sell
            if not self.position:  # And we do not already have an open position
                self.enter_short()  # Enter a short position
        # If the signal is 0, do nothing (no trade)

    def enter_long(self):
        entry_price = self.data.Close[-1]
        stop_loss = entry_price * (1 - self.risk_per_trade)
        take_profit = entry_price * (1 + self.reward_to_risk_ratio * self.risk_per_trade)

        # Calculate position size based on the stop loss
        amount_to_risk = self.equity * self.risk_per_trade
        risk_per_share = entry_price - stop_loss
        size_in_units = amount_to_risk / risk_per_share

        # Make sure size is a whole number of shares/units
        size_in_units = max(1, int(size_in_units))

        # Enter a long position with the calculated size
        self.buy(size=size_in_units, tp=take_profit, sl=stop_loss)

    def enter_short(self):
        entry_price = self.data.Close[-1]
        stop_loss = entry_price * (1 + self.risk_per_trade)
        take_profit = entry_price * (1 - self.reward_to_risk_ratio * self.risk_per_trade)

        # Calculate position size based on the stop loss
        amount_to_risk = self.equity * self.risk_per_trade
        risk_per_share = stop_loss - entry_price
        size_in_units = amount_to_risk / risk_per_share

        # Make sure size is a whole number of shares/units
        size_in_units = max(1, int(size_in_units))

        # Enter a short position with the calculated size
        self.sell(size=size_in_units, tp=take_profit, sl=stop_loss)

In [ ]:
bt = Backtest(df, SignalStrategy, cash=100000, commission=.00001, exclusive_orders=True)
output = bt.run()

# Now, you can print the overview of the backtest results
print(output)

# To get a DataFrame with all closed trades
trades = output._trades

# Show the trades DataFrame
print(trades)

Start                     2023-09-12 19:00...
End                       2023-12-29 21:45...
Duration                    108 days 02:45:00
Exposure Time [%]                   99.717075
Equity Final [$]                 113832.66419
Equity Peak [$]                 117348.155147
Return [%]                          13.832664
Buy & Hold Return [%]                7.832427
Return (Ann.) [%]                   42.176814
Volatility (Ann.) [%]               16.756349
Sharpe Ratio                         2.517065
Sortino Ratio                         7.30457
Calmar Ratio                         9.852151
Max. Drawdown [%]                   -4.280975
Avg. Drawdown [%]                    -0.35705
Max. Drawdown Duration       42 days 03:00:00
Avg. Drawdown Duration        1 days 03:04:00
# Trades                                   47
Win Rate [%]                        63.829787
Best Trade [%]                       1.004587
Worst Trade [%]                     -1.631291
Avg. Trade [%]                    

In [ ]:
bt.plot()

C:\Users\zakii\Documents\FX_Algo\Code\venv\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\zakii\Documents\FX_Algo\Code\venv\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\zakii\Documents\FX_Algo\Code\venv\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\zakii\Documents\FX_Algo\Code\venv\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2490', ...)

In [ ]:
trades_df = stats['_trades']
print(trades_df)